In [132]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [133]:
train=pd.read_csv(r'C:\Users\enesm\OneDrive\Masaüstü\KAGGLE\GDZ\Data\Cleaned\train_v2.csv')
holidays=pd.read_csv(r'C:\Users\enesm\OneDrive\Masaüstü\KAGGLE\GDZ\Data\Cleaned\holidays_v2.csv')
test=pd.read_csv(r'C:\Users\enesm\OneDrive\Masaüstü\KAGGLE\GDZ\Data\Cleaned\test_v2.csv')
weather=pd.read_csv(r'C:\Users\enesm\OneDrive\Masaüstü\KAGGLE\GDZ\Data\Cleaned\weather_v2.csv')
sample_submission=pd.read_csv(r'C:\Users\enesm\OneDrive\Masaüstü\KAGGLE\GDZ\Data\Raw\sample_submission.csv')

In [134]:
def make_datetime(df,column):
    df[column]=pd.to_datetime(df[column])
    return df

In [135]:
def flag_ifexist(df,column):
    df[column]=df[column].apply( lambda x : 1 if pd.notna(x) else 0)
    return df

In [136]:
train=make_datetime(train,'tarih')
test=make_datetime(test,'tarih')
weather=make_datetime(weather,'date')
holidays=make_datetime(holidays,'Date')

In [137]:
weather=weather.rename({'ilce_0_first':'ilce_0','ilce_1_first':'ilce_1'},axis=1)
holidays=holidays.rename({'Date':'tarih'},axis=1)
weather=weather.rename({'date':'tarih'},axis=1)
train_merged=pd.merge(train,weather,on=['tarih','ilce_0','ilce_1'],how='left')
holidays=holidays.drop(columns=holidays.columns[:3],axis=0)

In [138]:
def merge_holiday_weather(df):
    df=pd.merge(df,weather,on=['tarih','ilce_0','ilce_1'],how='left')
    df=pd.merge(df,holidays,on='tarih',how='left')
    df['Bayram_Flag'].apply(lambda x : 1 if pd.notna(x) else x)
    df=flag_ifexist(df,'Bayram_Flag')
    return df 

In [139]:
train=merge_holiday_weather(train)
train

,tarih,ilce_0,ilce_1,bildirimsiz_sum,bildirimli_sum,t_2m:C_min,t_2m:C_max,t_2m:C_mean,effective_cloud_cover:p_min,effective_cloud_cover:p_max,...,wind_speed_10m:ms_min,wind_speed_10m:ms_max,wind_speed_10m:ms_mean,prob_precip_1h:p_min,prob_precip_1h:p_max,prob_precip_1h:p_mean,t_apparent:C_min,t_apparent:C_max,t_apparent:C_mean,Bayram_Flag
0,2021-01-01,manisa,akhisar,9,0,9.0,14.9,11.133333,30.5,75.0,...,0.2,2.1,1.333333,1.0,1.0,1.000000,8.4,17.8,11.837500,1
1,2021-01-02,manisa,akhisar,10,0,6.4,17.1,10.050000,3.3,49.6,...,0.4,1.4,1.108333,1.0,1.0,1.000000,5.7,20.2,10.904167,0
2,2021-01-03,manisa,akhisar,5,2,5.0,14.5,8.341667,1.3,99.8,...,0.2,1.7,0.966667,1.0,1.0,1.000000,3.6,16.3,8.587500,0
3,2021-01-04,manisa,akhisar,6,0,5.4,17.5,10.370833,7.7,95.5,...,0.5,2.7,1.612500,1.0,30.4,2.295833,4.3,19.2,10.670833,0
4,2021-01-05,manisa,akhisar,15,4,8.7,15.6,11.916667,8.0,100.0,...,0.9,3.1,1.837500,1.0,53.9,6.770833,8.6,18.6,12.604167,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46939,2023-12-28,izmir,selcuk,1,0,7.8,18.8,12.237500,0.0,92.4,...,1.0,2.5,1.462500,1.0,1.0,1.000000,7.8,22.1,13.295833,0
46940,2023-12-29,izmir,selcuk,1,1,7.3,18.3,11.970833,0.0,48.5,...,0.2,1.5,1.008333,1.0,1.0,1.000000,6.5,22.7,13.175000,0
46941,2023-12-30,izmir,selcuk,0,2,8.4,18.1,12.383333,5.2,92.0,...,0.7,2.6,1.233333,1.0,1.0,1.000000,8.4,21.0,13.437500,0
46942,2023-12-31,izmir,selcuk,1,0,9.9,18.7,13.479167,18.3,87.3,...,0.7,2.1,1.250000,1.0,1.0,1.000000,9.7,22.5,14.725000,0


# Test Preperation

In [140]:
test=merge_holiday_weather(test)

In [147]:
train.ilce_0=train.ilce_0.apply(lambda x:1 if x=='manisa' else 0)
test.ilce_0=test.ilce_0.apply(lambda x:1 if x=='manisa' else 0)

# Eda

In [153]:
from sklearn.preprocessing import LabelEncoder

In [156]:
encoder=LabelEncoder()
train['ilce_1']=encoder.fit_transform(train['ilce_1'])
test['ilce_1']=encoder.transform(test['ilce_1'])

In [157]:
train

,tarih,ilce_0,ilce_1,bildirimsiz_sum,bildirimli_sum,t_2m:C_min,t_2m:C_max,t_2m:C_mean,effective_cloud_cover:p_min,effective_cloud_cover:p_max,...,wind_speed_10m:ms_min,wind_speed_10m:ms_max,wind_speed_10m:ms_mean,prob_precip_1h:p_min,prob_precip_1h:p_max,prob_precip_1h:p_mean,t_apparent:C_min,t_apparent:C_max,t_apparent:C_mean,Bayram_Flag
0,2021-01-01,1,1,9,0,9.0,14.9,11.133333,30.5,75.0,...,0.2,2.1,1.333333,1.0,1.0,1.000000,8.4,17.8,11.837500,1
1,2021-01-02,1,1,10,0,6.4,17.1,10.050000,3.3,49.6,...,0.4,1.4,1.108333,1.0,1.0,1.000000,5.7,20.2,10.904167,0
2,2021-01-03,1,1,5,2,5.0,14.5,8.341667,1.3,99.8,...,0.2,1.7,0.966667,1.0,1.0,1.000000,3.6,16.3,8.587500,0
3,2021-01-04,1,1,6,0,5.4,17.5,10.370833,7.7,95.5,...,0.5,2.7,1.612500,1.0,30.4,2.295833,4.3,19.2,10.670833,0
4,2021-01-05,1,1,15,4,8.7,15.6,11.916667,8.0,100.0,...,0.9,3.1,1.837500,1.0,53.9,6.770833,8.6,18.6,12.604167,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46939,2023-12-28,0,39,1,0,7.8,18.8,12.237500,0.0,92.4,...,1.0,2.5,1.462500,1.0,1.0,1.000000,7.8,22.1,13.295833,0
46940,2023-12-29,0,39,1,1,7.3,18.3,11.970833,0.0,48.5,...,0.2,1.5,1.008333,1.0,1.0,1.000000,6.5,22.7,13.175000,0
46941,2023-12-30,0,39,0,2,8.4,18.1,12.383333,5.2,92.0,...,0.7,2.6,1.233333,1.0,1.0,1.000000,8.4,21.0,13.437500,0
46942,2023-12-31,0,39,1,0,9.9,18.7,13.479167,18.3,87.3,...,0.7,2.1,1.250000,1.0,1.0,1.000000,9.7,22.5,14.725000,0


In [158]:
test

,tarih,ilce_0,ilce_1,bildirimli_sum,t_2m:C_min,t_2m:C_max,t_2m:C_mean,effective_cloud_cover:p_min,effective_cloud_cover:p_max,effective_cloud_cover:p_mean,...,wind_speed_10m:ms_min,wind_speed_10m:ms_max,wind_speed_10m:ms_mean,prob_precip_1h:p_min,prob_precip_1h:p_max,prob_precip_1h:p_mean,t_apparent:C_min,t_apparent:C_max,t_apparent:C_mean,Bayram_Flag
0,2024-01-04,1,1,0,8.4,17.0,12.425000,0.0,95.5,38.841667,...,0.4,2.9,1.437500,1.0,1.0,1.000000,7.9,19.4,13.358333,0
1,2024-01-04,1,2,1,8.9,17.5,12.912500,9.7,99.9,59.437500,...,0.1,2.3,1.450000,1.0,1.0,1.000000,8.6,19.8,13.875000,0
2,2024-01-04,1,13,0,7.2,11.1,8.620833,37.7,100.0,83.816667,...,1.0,3.6,2.045833,1.0,1.0,1.000000,6.0,13.7,8.712500,0
3,2024-01-04,1,17,0,9.2,17.7,13.187500,1.0,98.6,46.291667,...,0.3,4.3,2.166667,1.0,1.0,1.000000,8.7,19.7,13.750000,0
4,2024-01-04,1,18,0,6.3,13.3,9.441667,36.3,100.0,76.104167,...,0.1,4.6,1.783333,1.0,1.0,1.000000,6.5,15.2,9.904167,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,2024-01-31,0,4,0,4.5,11.0,7.133333,0.0,95.1,19.504167,...,2.1,5.8,3.612500,1.0,1.0,1.000000,1.5,15.2,6.495833,0
1312,2024-01-31,0,8,0,1.6,10.8,5.670833,0.0,75.5,22.491667,...,0.5,3.0,1.845833,1.0,1.0,1.000000,-0.6,15.1,5.950000,0
1313,2024-01-31,0,32,0,4.8,11.5,7.466667,0.0,94.4,16.250000,...,2.0,6.0,3.791667,1.0,5.9,1.204167,1.7,15.8,6.866667,0
1314,2024-01-31,1,0,0,-1.4,8.8,2.841667,0.0,13.9,2.883333,...,1.0,4.3,2.470833,1.0,1.0,1.000000,-4.0,11.4,2.166667,0


In [151]:
test.ilce_1.unique()

array(['akhisar', 'alasehir', 'demirci', 'golmarmara', 'gordes',
       'kirkagac', 'koprubasi', 'salihli', 'sarigol', 'saruhanli',
       'selendi', 'soma', 'turgutlu', 'yunusemre', 'sehzadeler', 'aliaga',
       'bayindir', 'bayrakli', 'bergama', 'bornova', 'buca', 'dikili',
       'foca', 'gaziemir', 'guzelbahce', 'karabaglar', 'karaburun',
       'karsiyaka', 'kemalpasa', 'kinik', 'konak', 'kiraz', 'menderes',
       'menemen', 'seferihisar', 'torbali', 'tire', 'urla', 'cesme',
       'cigli', 'odemis', 'kula', 'balcova', 'beydag', 'narlidere',
       'ahmetli', 'selcuk'], dtype=object)

In [169]:
train=train.sort_values(by='tarih',ascending=True).reset_index(drop=True)

In [170]:
train.to_csv('train_merged.csv',index=False)
test.to_csv('test_merged.csv',index=False)

# Train Simple Random Forest

In [159]:
from sklearn.ensemble import RandomForestRegressor

In [161]:
rand_forest=RandomForestRegressor()


,tarih,ilce_0,ilce_1,bildirimsiz_sum,bildirimli_sum,t_2m:C_min,t_2m:C_max,t_2m:C_mean,effective_cloud_cover:p_min,effective_cloud_cover:p_max,...,wind_speed_10m:ms_min,wind_speed_10m:ms_max,wind_speed_10m:ms_mean,prob_precip_1h:p_min,prob_precip_1h:p_max,prob_precip_1h:p_mean,t_apparent:C_min,t_apparent:C_max,t_apparent:C_mean,Bayram_Flag
0,2021-01-01,1,1,9,0,9.0,14.9,11.133333,30.5,75.0,...,0.2,2.1,1.333333,1.0,1.0,1.000000,8.4,17.8,11.837500,1
1,2021-01-02,1,1,10,0,6.4,17.1,10.050000,3.3,49.6,...,0.4,1.4,1.108333,1.0,1.0,1.000000,5.7,20.2,10.904167,0
2,2021-01-03,1,1,5,2,5.0,14.5,8.341667,1.3,99.8,...,0.2,1.7,0.966667,1.0,1.0,1.000000,3.6,16.3,8.587500,0
3,2021-01-04,1,1,6,0,5.4,17.5,10.370833,7.7,95.5,...,0.5,2.7,1.612500,1.0,30.4,2.295833,4.3,19.2,10.670833,0
4,2021-01-05,1,1,15,4,8.7,15.6,11.916667,8.0,100.0,...,0.9,3.1,1.837500,1.0,53.9,6.770833,8.6,18.6,12.604167,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46939,2023-12-28,0,39,1,0,7.8,18.8,12.237500,0.0,92.4,...,1.0,2.5,1.462500,1.0,1.0,1.000000,7.8,22.1,13.295833,0
46940,2023-12-29,0,39,1,1,7.3,18.3,11.970833,0.0,48.5,...,0.2,1.5,1.008333,1.0,1.0,1.000000,6.5,22.7,13.175000,0
46941,2023-12-30,0,39,0,2,8.4,18.1,12.383333,5.2,92.0,...,0.7,2.6,1.233333,1.0,1.0,1.000000,8.4,21.0,13.437500,0
46942,2023-12-31,0,39,1,0,9.9,18.7,13.479167,18.3,87.3,...,0.7,2.1,1.250000,1.0,1.0,1.000000,9.7,22.5,14.725000,0


In [163]:
sample_submission

,unique_id,bildirimsiz_sum
0,2024-01-04-manisa-akhisar,NaN
1,2024-01-04-manisa-alasehir,NaN
2,2024-01-04-manisa-demirci,NaN
3,2024-01-04-manisa-golmarmara,NaN
4,2024-01-04-manisa-gordes,NaN
...,...,...
1311,2024-01-31-izmir-balcova,NaN
1312,2024-01-31-izmir-beydag,NaN
1313,2024-01-31-izmir-narlidere,NaN
1314,2024-01-31-manisa-ahmetli,NaN
